### Decision Tree C4.5

In [1]:
import pandas as pd
import numpy as np

In [2]:
DF = pd.read_csv('Naive_Labelled.csv')
DF

,Age,Income,Student,Credit_Rating,Actual
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [3]:
DF.describe()

,Age,Income,Student,Credit_Rating,Actual
count,14,14,14,14,14
unique,3,3,2,2,2
top,youth,medium,no,fair,yes
freq,5,6,7,8,9


In [4]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Age            14 non-null     object
 1   Income         14 non-null     object
 2   Student        14 non-null     object
 3   Credit_Rating  14 non-null     object
 4   Actual         14 non-null     object
dtypes: object(5)
memory usage: 692.0+ bytes


#### Node class

In [5]:
class Node:
    def __init__(self,attribute):
        self.attribute: str = attribute
        self.node_dict = {}

#### Decision Tree Class

In [50]:
class DecisionTree:
    def __init__ (self):
        self.root: Node = None
        self.info_D = 0

    def fit(self,df: pd.DataFrame):
        self.df = df
        # Find the uniue Target classes..
        self.Target_classes = df['Actual'].unique()
        # Calculate Info for each attribute..
        for class_ in self.Target_classes:
            p_i = len(df[df['Actual']==class_])/len(df)
            self.info_D+= -(p_i * np.log2(p_i))

        print('info_D: ',self.info_D)

        # Set attribute for root and pass it to the recursive fucntion
        self.root = Node(self.getAttribute(df))
        self.ConstructTree(self.root,df)

    def ConstructTree(self,node: Node,df: pd.DataFrame):
        if node.attribute in self.Target_classes:
            return node

        print('Node: ',node.attribute)
        for category in df[node.attribute].unique():
            # Get node / attribute for each category...
            attrib = self.getAttribute(df[df[node.attribute]==category].drop(columns = node.attribute))
            node.node_dict[category] = self.ConstructTree(Node(attrib),df[df[node.attribute]==category].drop(columns = node.attribute))
            print('category: ',category)
            print('attrib: ',node.node_dict[category].attribute)
            # node.edgeName.append(category)

        return node # Should return this node in order to connect the node with the parent...

    def getAttribute(self,df: pd.DataFrame)-> str:
        # if there is only one class in Target column then return that class
        cat = df['Actual'].unique()
        if len(cat)==1:
            return cat[0]

        attribute_gain_dict = {}
        attribute_gain_ratio = {}
        # Calculating info for each attribute...
        for feature in df:
            # for each feature
            if feature != 'Actual':
                info = 0
                split_info = 0
                # for each attribute in feature...
                for category in df[feature].unique():
                    p_i = len(df[df[feature]==category])/len(df) # common for info and split_info
                    info +=  p_i * self.info_dj(feature,category,df)
                    if p_i!=0:
                        split_info+= -(p_i * np.log2(p_i))

                attribute_gain_dict[feature] = info
                attribute_gain_ratio[feature] = split_info


        # Calculating gain for each attribute..
        attribute_gain_dict = {key: self.info_D-value for key,value in attribute_gain_dict.items()}
        # Calculating gain ratio for each attribute..
        attribute_gain_ratio = {key: attribute_gain_dict[key]/value for key,value in attribute_gain_ratio.items()}

        # Select the attribute with highest gain_ratio....        
        return max(attribute_gain_ratio, key = lambda x: attribute_gain_ratio[x])

    def info_dj(self,feature: str,category: str,df: pd.DataFrame):
        res = 0
        # find info_Dj 
        for class_ in self.Target_classes:
            p_i = len(df[(df[feature]==category) & (df['Actual']==class_)])/len(df[df[feature]==category])
            if p_i !=0:
               res+= -(p_i * np.log2(p_i))     

        return res
    

In [51]:
model = DecisionTree()
model.fit(DF.copy())

info_D:  0.9402859586706311
Node:  Age
Node:  Student 
category:  no
attrib:  no
category:  yes
attrib:  yes
category:  youth
attrib:  Student 
category:  middle aged
attrib:  yes
Node:  Credit_Rating
category:  fair
attrib:  yes
category:  excellent
attrib:  no
category:  senior
attrib:  Credit_Rating
